In [6]:
from vanna.ollama import Ollama
from pymilvus import MilvusClient, model
from vanna.milvus import Milvus_VectorStore
import pyodbc
from pymilvus import MilvusClient
from sentence_transformers import SentenceTransformer
import logging
from datetime import datetime
from typing import List
import requests
import json

# Configure logging with more detailed format
logging.basicConfig(
    level=logging.DEBUG,  # Changed to DEBUG level for more detailed logs
    format='%(asctime)s [%(levelname)s] - %(message)s',
    handlers=[
        logging.FileHandler(f'mssql_ddl_training_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

# Load model
model = SentenceTransformer("TencentBAC/Conan-embedding-v1")

class CustomEmbeddingFunction:
    def __init__(self, model=model):
        self.model = model
        logging.info("CustomEmbeddingFunction initialized")
        
    def __call__(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        logging.debug(f"Processing texts in __call__: {texts[:100]}...")  # Log first 100 chars
        embeddings = self.model.encode(texts, normalize_embeddings=True)
        logging.debug(f"Generated embeddings shape: {embeddings.shape}")
        return embeddings
    
    def encode_documents(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        logging.debug(f"Encoding documents: {texts[:100]}...")
        return self.model.encode(texts, normalize_embeddings=True)
    
    def encode_queries(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        logging.debug(f"Encoding queries: {texts[:100]}...")
        return self.model.encode(texts, normalize_embeddings=True)

# Milvus setup
milvus_uri = "http://10.13.18.40:19530"
milvus_client = MilvusClient(uri=milvus_uri, db_name='SQLRAG_YM_ALMS')

class MyVanna(Milvus_VectorStore, Ollama):
    def __init__(self, config=None):
        Milvus_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)
        logging.info("MyVanna initialized with config")

    def train(self, ddl=None, documentation=None):  # 添加 documentation 參數
        try:
            if ddl is not None:
                logging.info(f"Starting training with DDL: {ddl[:100]}...")
                super().train(ddl=ddl)
                logging.info("DDL training completed successfully")
            
            if documentation is not None:
                logging.info(f"Starting training with documentation: {documentation[:100]}...")
                super().train(documentation=documentation)
                logging.info("Documentation training completed successfully")
                
        except Exception as e:
            logging.error(f"Error during training: {str(e)}")
            raise

# Initialize vn_milvus
vn_milvus = MyVanna(config={
    'model': 'mistral-nemo',
    'milvus_client': milvus_client,
    "embedding_function": CustomEmbeddingFunction(),
    "n_results": 3,
})

# Database connection
db = 'YM_ALMS'
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=UTCYMEDCLSNR01;'
    f'DATABASE={db};'
    'UID=ALS2.0_Admin_readonly;'
    'PWD=5rdxZSE$;'
    'Trusted_Connection=no;'
    'Encrypt=yes;'
    'TrustServerCertificate=yes;'
)
cursor = conn.cursor()

def get_table_ddl_improved(cursor, table_name: str) -> str:
    """
    Get DDL for a specific table with improved query
    """
    try:
        # First get the schema information
        cursor.execute(f"""
            SELECT 
                COLUMN_NAME,
                DATA_TYPE,
                CHARACTER_MAXIMUM_LENGTH,
                IS_NULLABLE,
                COLUMN_DEFAULT
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = '{table_name}'
            ORDER BY ORDINAL_POSITION
        """)
        
        columns = cursor.fetchall()
        
        # Build CREATE TABLE statement
        ddl = f"CREATE TABLE [{table_name}] (\n"
        
        for i, col in enumerate(columns):
            col_name, data_type, max_length, is_nullable, default = col
            
            # Build column definition
            col_def = f"    [{col_name}] {data_type}"
            
            if max_length and max_length != -1:
                col_def += f"({max_length})"
            elif max_length == -1:
                col_def += "(MAX)"
                
            if is_nullable == 'NO':
                col_def += " NOT NULL"
            else:
                col_def += " NULL"
                
            if default:
                col_def += f" DEFAULT {default}"
                
            if i < len(columns) - 1:
                col_def += ","
                
            ddl += col_def + "\n"
        
        ddl += ")"
        
        # Get primary key information
        cursor.execute(f"""
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
            WHERE TABLE_NAME = '{table_name}'
            AND CONSTRAINT_NAME IN (
                SELECT CONSTRAINT_NAME 
                FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS 
                WHERE TABLE_NAME = '{table_name}' 
                AND CONSTRAINT_TYPE = 'PRIMARY KEY'
            )
        """)
        
        pk_columns = cursor.fetchall()
        if pk_columns:
            pk_cols = ", ".join([f"[{col[0]}]" for col in pk_columns])
            ddl += f",\n    CONSTRAINT [PK_{table_name}] PRIMARY KEY ({pk_cols})"
        
        logging.info(f"Generated DDL for table {table_name}: {ddl[:200]}...")  # Log first 200 chars
        return ddl
    
    except Exception as e:
        logging.error(f"Failed to get DDL for table {table_name}: {str(e)}")
        raise

def get_all_tables(cursor) -> List[str]:
    """
    Get all table names from the database
    """
    try:
        cursor.execute("""
            SELECT TABLE_NAME 
            FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_TYPE = 'BASE TABLE'
            AND TABLE_SCHEMA = 'dbo'  -- Only get tables from dbo schema
        """)
        tables = [row[0] for row in cursor.fetchall()]
        logging.info(f"Found {len(tables)} tables: {tables}")
        return tables
    except Exception as e:
        logging.error(f"Failed to get tables: {str(e)}")
        raise

def generate_text(prompt: str, model: str = "mistral-nemo") -> str:
    try:
        # 發送請求到 Ollama
        response = requests.post(
            'http://localhost:11434/api/generate',
            json={
                "model": model,
                "prompt": prompt
            },
            stream=True  # 啟用串流模式來處理回應
        )
        
        # 檢查請求是否成功
        response.raise_for_status()
        
        # 收集完整的回應
        full_response = ""
        for line in response.iter_lines():
            if line:
                # 解析每一行的 JSON
                json_response = json.loads(line.decode())
                if 'response' in json_response:
                    full_response += json_response['response']
                
        return full_response
        
    except requests.exceptions.RequestException as e:
        return f"網路請求錯誤: {str(e)}"
    except json.JSONDecodeError as e:
        return f"JSON 解析錯誤: {str(e)}"
    except Exception as e:
        return f"發生錯誤: {str(e)}"

def process_tables():
    """
    Main function to process all tables
    """
    try:
        # Get all tables
        tables = get_all_tables(cursor)
        total_tables = len(tables)
        
        # Process each table
        for idx, table_name in enumerate(tables, 1):
            try:
                logging.info(f"Processing table {idx}/{total_tables}: {table_name}")
                
                # Get DDL for the table
                ddl = get_table_ddl_improved(cursor, table_name)
                
                if not ddl:
                    logging.warning(f"No DDL generated for table: {table_name}")
                    continue

                # prompt = f"""請依據以下規則說明這段DDL:
                #             1.說明表名和每一個欄位名的意思，資料型態不用說明
                #             2.不要列點，以一段話完成，不要分段
                #             3.請使用繁體中文回覆並依照格式: XXX資料表用來儲存YYY，其中ZZZ欄位代表...
                #             {ddl}"""
                # response = generate_text(prompt)
                # vn_milvus.train(documentation=f"{response}")
                
                # Train the model with the DDL
                logging.info(f"Starting training for table: {table_name}")
                vn_milvus.train(ddl=ddl)

                prompt = f"""請依據以下規則說明這段DDL:
                            1.說明表名和每一個欄位名的意思，資料型態不用說明
                            2.不要列點，以一段話完成，不要分段
                            3.請使用繁體中文回覆並依照格式: XXX資料表用來儲存YYY，其中ZZZ欄位代表...
                            {ddl}"""
                response = generate_text(prompt)
                vn_milvus.train(documentation=response)

                logging.info(f"Successfully completed training for table: {table_name}")
                
            except Exception as e:
                logging.error(f"Error processing table {table_name}: {str(e)}")
                continue

    except Exception as e:
        logging.error(f"Main process error: {str(e)}")
    
    finally:
        cursor.close()
        conn.close()
        logging.info("Database connections closed")



if __name__ == "__main__":
    logging.info(f"Starting DDL extraction and training process at {datetime.now()}")
    start_time = datetime.now()
    
    # 檢查 Milvus 連接
    try:
        collections = milvus_client.list_collections()
        logging.info(f"Current Milvus collections: {collections}")
    except Exception as e:
        logging.error(f"Error connecting to Milvus: {str(e)}")
        raise
    
    process_tables()
    
    # 再次檢查 Milvus 集合
    try:
        collections = milvus_client.list_collections()
        logging.info(f"Milvus collections after processing: {collections}")
        
        # 檢查每個集合的實體數量
        for collection in collections:
            count = milvus_client.get_collection_stats(collection)
            logging.info(f"Collection {collection} stats: {count}")
    except Exception as e:
        logging.error(f"Error checking Milvus collections: {str(e)}")
    
    end_time = datetime.now()
    duration = end_time - start_time
    logging.info(f"Process completed. Total duration: {duration}")

2025-04-02 06:45:30,771 [INFO] - Use pytorch device_name: cuda
2025-04-02 06:45:30,772 [INFO] - Load pretrained SentenceTransformer: TencentBAC/Conan-embedding-v1
2025-04-02 06:45:30,773 [DEBUG] - Starting new HTTPS connection (1): huggingface.co:443
2025-04-02 06:45:31,008 [DEBUG] - https://huggingface.co:443 "HEAD /TencentBAC/Conan-embedding-v1/resolve/main/modules.json HTTP/11" 200 0
2025-04-02 06:45:31,213 [DEBUG] - https://huggingface.co:443 "HEAD /TencentBAC/Conan-embedding-v1/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
2025-04-02 06:45:31,408 [DEBUG] - https://huggingface.co:443 "HEAD /TencentBAC/Conan-embedding-v1/resolve/main/README.md HTTP/11" 200 0
2025-04-02 06:45:31,598 [DEBUG] - https://huggingface.co:443 "HEAD /TencentBAC/Conan-embedding-v1/resolve/main/modules.json HTTP/11" 200 0
2025-04-02 06:45:31,844 [DEBUG] - https://huggingface.co:443 "HEAD /TencentBAC/Conan-embedding-v1/resolve/main/sentence_bert_config.json HTTP/11" 200 0
2025-04-02 06:45:32,040

Adding ddl: CREATE TABLE [OCR_0220_Recognition] (
    [factory] varchar(3) NOT NULL,
    [customerID] int NOT NULL,
    [PartNum] varchar(20) NOT NULL,
    [RecordTime] datetime NULL,
    [Status] varchar(15) NOT NULL,
    [customer_xml] xml(MAX) NULL,
    [cam_xml] xml(MAX) NULL,
    [answer_xml] xml(MAX) NULL
),
    CONSTRAINT [PK_OCR_0220_Recognition] PRIMARY KEY ([customerID], [factory], [PartNum])


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.08it/s]
2025-04-02 06:45:53,815 [INFO] - DDL training completed successfully
2025-04-02 06:45:53,817 [DEBUG] - Starting new HTTP connection (1): localhost:11434
2025-04-02 06:45:57,137 [DEBUG] - http://localhost:11434 "POST /api/generate HTTP/11" 200 None
2025-04-02 06:45:58,862 [INFO] - Starting training with documentation: OCR_0220_Recognition資料表用來儲存Optical Character Recognition（光學字元辨識）的結果，其中factory欄位代表工廠代碼，長度為3；customerI...
2025-04-02 06:45:58,863 [DEBUG] - Encoding documents: ['OCR_0220_Recognition資料表用來儲存Optical Character Recognition（光學字元辨識）的結果，其中factory欄位代表工廠代碼，長度為3；customerID欄位代表客戶ID，使用整數型態；PartNum欄位代表零件序號，長度為20；RecordTime欄位記錄辨識時間，允許空值；Status欄位代表辨識狀態，長度為15，不允許空值；customer_xml、cam_xml和answer_xml三個欄位則用來存放客戶、相機和答案的XML資料，這三個欄位允許空值。主鍵由customerID、factory和PartNum三個欄位組成。']...


Adding documentation....


Batches: 100%|██████████| 1/1 [00:00<00:00, 91.77it/s]
2025-04-02 06:45:58,887 [INFO] - Documentation training completed successfully
2025-04-02 06:45:58,888 [INFO] - Successfully completed training for table: OCR_0220_Recognition
2025-04-02 06:45:58,888 [INFO] - Processing table 2/50: OCR_0220_Config
2025-04-02 06:45:59,025 [INFO] - Generated DDL for table OCR_0220_Config: CREATE TABLE [OCR_0220_Config] (
    [factory] varchar(3) NOT NULL,
    [mode] nvarchar(100) NOT NULL,
    [customerID] int NOT NULL,
    [Category] varchar(20) NOT NULL,
    [angle] varchar(20) NOT N...
2025-04-02 06:45:59,026 [INFO] - Starting training for table: OCR_0220_Config
2025-04-02 06:45:59,026 [INFO] - Starting training with DDL: CREATE TABLE [OCR_0220_Config] (
    [factory] varchar(3) NOT NULL,
    [mode] nvarchar(100) NOT NUL...
2025-04-02 06:45:59,027 [DEBUG] - Encoding documents: ['CREATE TABLE [OCR_0220_Config] (\n    [factory] varchar(3) NOT NULL,\n    [mode] nvarchar(100) NOT NULL,\n    [customerI

...

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.81it/s]
2025-04-02 06:47:04,181 [INFO] - Documentation training completed successfully
2025-04-02 06:47:04,182 [INFO] - Successfully completed training for table: gai_tbl_servicesName
2025-04-02 06:47:04,182 [INFO] - Processing table 50/50: gai_tbl_service_models
2025-04-02 06:47:04,332 [INFO] - Generated DDL for table gai_tbl_service_models: CREATE TABLE [gai_tbl_service_models] (
    [model_id] bigint NOT NULL,
    [model_name] nvarchar(50) NOT NULL,
    [description] nvarchar(100) NULL,
    [created_time] datetime2 NULL,
    [updated_ti...
2025-04-02 06:47:04,333 [INFO] - Starting training for table: gai_tbl_service_models
2025-04-02 06:47:04,333 [INFO] - Starting training with DDL: CREATE TABLE [gai_tbl_service_models] (
    [model_id] bigint NOT NULL,
    [model_name] nvarchar(50...
2025-04-02 06:47:04,334 [DEBUG] - Encoding documents: ['CREATE TABLE [gai_tbl_service_models] (\n    [model_id] bigint NOT NULL,\n    [model_name] nvarcha

Adding ddl: CREATE TABLE [gai_tbl_service_models] (
    [model_id] bigint NOT NULL,
    [model_name] nvarchar(50) NOT NULL,
    [description] nvarchar(100) NULL,
    [created_time] datetime2 NULL,
    [updated_time] datetime2 NULL,
    [updated_person] nvarchar(50) NOT NULL,
    [gpu] varchar(50) NULL,
    [host] varchar(100) NOT NULL,
    [owner] nvarchar(50) NOT NULL
),
    CONSTRAINT [PK_gai_tbl_service_models] PRIMARY KEY ([model_id])


Batches: 100%|██████████| 1/1 [00:00<00:00, 101.76it/s]
2025-04-02 06:47:04,354 [INFO] - DDL training completed successfully
2025-04-02 06:47:04,356 [DEBUG] - Starting new HTTP connection (1): localhost:11434
2025-04-02 06:47:04,410 [DEBUG] - http://localhost:11434 "POST /api/generate HTTP/11" 200 None
2025-04-02 06:47:05,691 [INFO] - Starting training with documentation: " gai_tbl_service_models資料表用來儲存服務模型相關資訊，其中model_id欄位代表模型的唯一識別碼、model_name欄位則是模型的名稱，description用於記錄模型...
2025-04-02 06:47:05,692 [DEBUG] - Encoding documents: ['" gai_tbl_service_models資料表用來儲存服務模型相關資訊，其中model_id欄位代表模型的唯一識別碼、model_name欄位則是模型的名稱，description用於記錄模型的描述。created_time和updated_time分別紀錄模型建立與更新的時間，而updated_person欄位則是記錄誰最 recent更新了模型。gpu欄位用來標識模型所需的GPU類型，host欄位則存放模型所在的主機名稱，owner欄位紀錄模型的擁有者。此外，model_id為該表的主鍵，不允許空值。']...


Adding documentation....


Batches: 100%|██████████| 1/1 [00:00<00:00, 95.41it/s]
2025-04-02 06:47:05,713 [INFO] - Documentation training completed successfully
2025-04-02 06:47:05,713 [INFO] - Successfully completed training for table: gai_tbl_service_models
2025-04-02 06:47:05,716 [INFO] - Database connections closed
2025-04-02 06:47:05,717 [INFO] - Milvus collections after processing: ['vannaddl', 'vannasql', 'vannadoc']
2025-04-02 06:47:05,719 [INFO] - Collection vannaddl stats: {'row_count': 0}
2025-04-02 06:47:05,720 [INFO] - Collection vannasql stats: {'row_count': 0}
2025-04-02 06:47:05,721 [INFO] - Collection vannadoc stats: {'row_count': 0}
2025-04-02 06:47:05,722 [INFO] - Process completed. Total duration: 0:01:21.499788
